In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

import os

os.environ['CUDA_VISIBLE_DEVICES'] = "5"

base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Step 2: Add special token
special_tokens_dict = {'additional_special_tokens': ['<pad>']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

# Resize the model embeddings to accommodate the new token
base_model.resize_token_embeddings(len(tokenizer))

# Step 3: Load local LoRA model
lora_model_path = "/shared/data3/yichi9/Mistral-7B-Instruct-v0.2-sayself-question"
lora_model = PeftModel.from_pretrained(base_model, lora_model_path)

# Step 4: Merge models
# This step involves merging the weights of the base model with the LoRA model
base_model = lora_model.merge_and_unload()

# Step 5: Save the merged model
output_path = "/shared/data3/yichi9/Mistral-7B-Instruct-v0.2-sayself-question-full"
base_model.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)

print("Model merged and saved to", output_path)